In [2]:
import numpy as np
import pandas as pd
from numba import njit 
from collections import namedtuple
# Item = namedtuple("Item", ['index', 'value', 'weight'])

In [ ]:
##for fun haha
# !pip install jupyterthemes
# !jt -t chesterish

In [40]:
with open('gc_100_9.txt') as input_data:
    lines= list(input_data)

    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))

In [8]:
test=[(1,2), (3,4)]

In [41]:
constraint_order=[]

##first degree constraint_order
for i in range(node_count):
    count=0
    for k in edges:
        if i in k:
            count+=1
    constraint_order.append((i, count))

## second degree constraint_order
second_constraint_order=[]
for i in range(node_count):
    count=0
    for k in edges:
        if i in k:
            for index in range(len(constraint_order)):
                if constraint_order[index][0]==k[1-k.index(i)]:
                    count+=constraint_order[index][1]
    second_constraint_order.append((i, count))

constraint_order=sorted(second_constraint_order, key=lambda x: x[1])


##sometimes second degree works better;
## generally more complex data, better second degree performs.


fixed_order=[]
for i in constraint_order:
    fixed_order.append(i[0])

combined_edges=[]
for i in range(node_count):
    close_nodes=[]
    for k in edges:
        if i in k:
            close_nodes.append(k[1-k.index(i)])
    combined_edges.append(close_nodes)
    
    
total_color=node_count-1  ### at most 
table=np.zeros((node_count, total_color))

# total_color=8
# table=np.zeros((node_count, total_color))

In [117]:
fixed_order

[0, 7, 16, 17, 12, 14, 1, 8, 5, 3, 15, 19, 6, 13, 18, 2, 4, 9, 11, 10]

In [42]:
##0: haven't tries
##1: decision
##-1: not feasible
while constraint_order:
    constraint_pair= constraint_order.pop()
    node=constraint_pair[0]
    available_color_list = np.where(table[node]==0)  ##find all available colors

    
    if 0 in table[node]:
        color=available_color_list[0][0]  ##find the first avaiable color
        table[node][color]=1   ## color this node
        ##for other attched nodes, they cannot use the same color
                
        check=True
        for other in combined_edges[node]:
            if 1 not in table[other]:
                if np.where(table[other]==0)[0].size==1 and np.where(table[other]==0)[0][0]==color:
                    check=False
        
        if check:
            for other in combined_edges[node]:
                if 1 not in table[other]:
                    table[other][color]=-1
        else:
            table[node][color]=-1
            constraint_order.append((node, _))
      
    

## This whole part is for feasbility checking
## for some reasons, it's not working

#     else:
#         ##recover the current node constraint
#         node_constraint= np.zeros(total_color)
#         previous_nodes= fixed_order[fixed_order.index(node)+1:]
#         for previous_node in previous_nodes:
#             node_constraint[np.where(table[previous_node]==1)[0][0]]=-1
#         table[node]=node_constraint

        
#         previous_node=fixed_order[fixed_order.index(node)+1]
#         previous_color=np.where(table[previous_node]==1)[0][0]
#         table[previous_node][previous_color]= -1
#         for other in combined_edges[previous_node]:
#             if 1 not in table[other]:
#                 adjust=False
#                 for k in previous_nodes[1:]:
#                     if np.where(table[k]==1)[0][0]==previous_color and other in combined_edges[k]:
#                         adjust=True
#                 if adjust:
#                     table[other][previous_color]=0
                
        
#         constraint_order.append(constraint_pair)
#         constraint_order.append((previous_node, _))
#         print('node', node)
#         print('previous node', previous_node)
#         print('previous_color', previous_color)
#         print(len(constraint_order), ' nodes left')
#     if node==24:
#         print()

    
#     if len(constraint_order)==node_count:
#         print('fail')
    
# #     color=available_color_list[0][0]  ##find the first avaiable color
# #     table[node][color]=1   ## color this node
# #     ##for other attched nodes, they cannot use the same color
# #     for other in combined_edges[node]:
# #         table[other][color]=-1

output=[]
for i in range(node_count):
    output.append(np.where(table[i]==1)[0][0])

total_color=max(output)+1    

In [43]:
total_color

45

In [197]:
## this part is for acheieving further optimality. Smaller number of colors
total_color=85
table=np.zeros((node_count, total_color))

# total_color=2
# table=np.zeros((node_count, total_color))

In [275]:
test

array([[0., 0., 0.],
       [1., 2., 3.]])

In [295]:
np.where(test[1]==0)[0].size

0

In [287]:
k=np.zeros(test.shape)

In [286]:
test.shape

(2, 3)

In [289]:
for i in range(len(test)):
    k[i]=test[i]

In [290]:
test[0]=[1,1,1]

In [292]:
k

array([[0., 0., 0.],
       [1., 2., 3.]])

In [262]:
output[66]

41

In [263]:
output[96]

40

In [6]:
len(set(output))

9

In [314]:
# output

In [215]:
total_color

121

## check the output

In [21]:
for (a,b) in edges:
    if output[a]==output[b]:
        print('fail', a, b)
print('success')

success


In [205]:
edges[2]

(0, 3)

In [74]:
table

array([[-1.,  1.,  0.],
       [ 1., -1.,  0.],
       [-1.,  1.,  0.],
       [-1.,  1.,  0.]])

In [75]:
np.where(table[node]==0)[0][0]

2

### First Version

In [ ]:
def solve_it(input_data):
    import numpy as np
    import pandas as pd
    from numba import njit 
    from collections import namedtuple

    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))

    # solution
    constraint_order=[]
    for i in range(node_count):
        count=0
        for k in edges:
            if i in k:
                count+=1
        constraint_order.append((i, count))

    constraint_order=sorted(constraint_order, key=lambda x: x[1])

    fixed_order=[]
    for i in constraint_order:
        fixed_order.append(i[0])

    combined_edges=[]
    for i in range(node_count):
        close_nodes=[]
        for k in edges:
            if i in k:
                close_nodes.append(k[1-k.index(i)])
        combined_edges.append(close_nodes)
    
    
    total_color=node_count-1  ### at most 
    table=np.zeros((node_count, total_color))



    ###loop and search
##0: haven't tries
##1: decision
##-1: not feasible
    while constraint_order:
        constraint_pair= constraint_order.pop()
        node=constraint_pair[0]
        current_node_constraint= table[node]
        available_color_list = np.where(table[node]==0)  ##find all available colors
    
        if 0 in table[node]:
            color=available_color_list[0][0]  ##find the first avaiable color
            table[node][color]=1   ## color this node
            ##for other attched nodes, they cannot use the same color
            for other in combined_edges[node]:
                if 1 not in table[other]:
                    table[other][color]=-1      
            
        else:
            previous_node=fixed_order[fixed_order.index(node)+1]
            previous_color=np.where(table[previous_node]==1)[0][0]
            table[previous_node][previous_color]= -1
            for other in combined_edges[previous_node]:
                if 1 not in table[other] and other!=node:
                    table[other][previous_color]=0
            table[node]=current_node_constraint
        
            constraint_order.append(constraint_pair)
            constraint_order.append((previous_node, _))


    output=[]
    for i in range(node_count):
        output.append(np.where(table[i]==1)[0][0])

    total_color=len(set(output))  


    # prepare the solution in the specified output format
    output_data = str(total_color) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, output))

    return output_data


import sys

if __name__ == '__main__':
    import sys
    if len(sys.argv) > 1:
        file_location = sys.argv[1].strip()
        with open(file_location, 'r') as input_data_file:
            input_data = input_data_file.read()
        print(solve_it(input_data))
    else:
        print('This test requires an input file.  Please select one from the data directory. (i.e. python solver.py ./data/gc_4_1)')

